In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_4/models/B5R2b5_AN_1FC_fold4_3.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold4_3.csv') #เปลี่ยน Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(7368, 22)
(7368, 22)
Normal:  (3991, 22)
Abnormal:  (3377, 22)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,...,tagName,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight
0,0,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,MildFattyLiver,https://irisprodseatraining.blob.core.windows....,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267
1,1,11,11,15,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,ModerateFattyLiver,https://irisprodseatraining.blob.core.windows....,257.0,103.0,538.0,634.0,0.256617,0.115278,0.537205,0.705051
2,2,17,17,15,P1,P1,Abnormal,AB03,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,SevereFattyLiver,https://irisprodseatraining.blob.core.windows....,0.0,184.0,573.0,751.0,0.000862,0.190025,0.638344,0.773737
3,3,21,21,15,P1,P1,Abnormal,AB04,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,Cirrhosis,https://irisprodseatraining.blob.core.windows....,278.0,87.0,515.0,496.0,0.278395,0.097096,0.515427,0.551515
4,4,23,23,15,P1,P1,Abnormal,AB05,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,PDF1,https://irisprodseatraining.blob.core.windows....,249.0,69.0,439.0,385.0,0.254909,0.076894,0.448980,0.428283


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 7368 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.04093226, -0.12505786,  0.4284896 , ...,  0.0108035 ,
        -0.04578006, -0.0658696 ],
       [-0.15834226, -0.12786947,  0.02440978, ..., -0.09727917,
        -0.00973756, -0.11962314],
       [-0.19653964, -0.11638605, -0.07645595, ..., -0.14906237,
         0.18745758, -0.17158099],
       ...,
       [ 0.78188062,  1.00162816, -0.07426403, ...,  0.81303197,
        -0.05008079,  0.74824166],
       [ 0.33599287,  0.78928822, -0.14627133, ...,  0.22233547,
        -0.02708252,  0.60923845],
       [ 0.93689191,  1.30642474, -0.05365535, ...,  0.70706016,
        -0.04737405,  0.97184992]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.040932,-0.125058,0.428490,0.032545,-0.213930,-0.184845,0.398899,-0.150903,-0.145638,0.049405,...,-0.158280,0.055231,-0.110891,-0.105580,-0.198868,-0.094820,-0.139208,0.010803,-0.045780,-0.065870
1,-0.158342,-0.127869,0.024410,0.183758,-0.195865,-0.191633,0.088111,-0.224864,-0.184908,-0.018059,...,-0.183029,-0.138348,0.078980,-0.158061,-0.189552,-0.208259,-0.126517,-0.097279,-0.009738,-0.119623
2,-0.196540,-0.116386,-0.076456,0.257657,-0.215408,-0.197021,0.073793,-0.207234,-0.207675,0.012348,...,-0.167818,-0.051777,-0.148312,-0.160919,-0.192510,-0.117608,-0.127628,-0.149062,0.187458,-0.171581
3,-0.187042,-0.132262,0.085123,0.521759,-0.224618,-0.168221,0.738727,-0.197021,-0.222505,0.140345,...,-0.174650,0.481474,0.288092,-0.151669,-0.213137,0.592069,0.636800,-0.131538,0.735004,-0.176337
4,-0.126015,-0.097393,-0.002105,0.147688,-0.213671,-0.221513,0.201299,-0.154483,-0.210006,-0.111963,...,-0.150404,0.004389,-0.159299,-0.154677,-0.177579,0.356640,0.041372,-0.161269,0.126126,-0.140248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7363,0.564706,0.516487,-0.149783,-0.177350,0.579261,0.649701,-0.084360,0.237743,0.575143,-0.185882,...,0.360595,-0.158747,0.049055,0.392033,0.718744,-0.174543,-0.084121,0.087163,0.001014,0.437538
7364,0.992621,0.983916,-0.128802,-0.075368,0.959859,1.135864,-0.139844,0.551593,0.717120,-0.186859,...,0.809735,-0.179962,0.085908,0.417597,1.130619,-0.181905,0.092592,0.830731,-0.120254,0.966349
7365,0.781881,1.001628,-0.074264,-0.168212,0.926772,1.090948,-0.187881,0.478824,0.945628,-0.196880,...,0.502278,-0.217931,-0.016066,0.823462,1.047661,-0.160435,-0.101736,0.813032,-0.050081,0.748242
7366,0.335993,0.789288,-0.146271,-0.110158,0.554049,0.712445,-0.146368,0.059334,0.558470,-0.073038,...,0.706782,-0.212964,0.001059,0.555337,0.503232,-0.209546,-0.089230,0.222335,-0.027083,0.609238


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.040932,-0.125058,0.428490,0.032545,-0.213930,-0.184845,0.398899,-0.150903,-0.145638,0.049405,...,-0.198868,-0.094820,-0.139208,0.010803,-0.045780,-0.065870,Abnormal,P1,FP-A,AB01
1,-0.158342,-0.127869,0.024410,0.183758,-0.195865,-0.191633,0.088111,-0.224864,-0.184908,-0.018059,...,-0.189552,-0.208259,-0.126517,-0.097279,-0.009738,-0.119623,Abnormal,P1,FP-A,AB02
2,-0.196540,-0.116386,-0.076456,0.257657,-0.215408,-0.197021,0.073793,-0.207234,-0.207675,0.012348,...,-0.192510,-0.117608,-0.127628,-0.149062,0.187458,-0.171581,Abnormal,P1,FP-A,AB03
3,-0.187042,-0.132262,0.085123,0.521759,-0.224618,-0.168221,0.738727,-0.197021,-0.222505,0.140345,...,-0.213137,0.592069,0.636800,-0.131538,0.735004,-0.176337,Abnormal,P1,FP-A,AB04
4,-0.126015,-0.097393,-0.002105,0.147688,-0.213671,-0.221513,0.201299,-0.154483,-0.210006,-0.111963,...,-0.177579,0.356640,0.041372,-0.161269,0.126126,-0.140248,Abnormal,P1,FP-A,AB05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7363,0.564706,0.516487,-0.149783,-0.177350,0.579261,0.649701,-0.084360,0.237743,0.575143,-0.185882,...,0.718744,-0.174543,-0.084121,0.087163,0.001014,0.437538,Normal,P8,FP-E,Normal
7364,0.992621,0.983916,-0.128802,-0.075368,0.959859,1.135864,-0.139844,0.551593,0.717120,-0.186859,...,1.130619,-0.181905,0.092592,0.830731,-0.120254,0.966349,Normal,P8,FP-E,Normal
7365,0.781881,1.001628,-0.074264,-0.168212,0.926772,1.090948,-0.187881,0.478824,0.945628,-0.196880,...,1.047661,-0.160435,-0.101736,0.813032,-0.050081,0.748242,Normal,P8,FP-E,Normal
7366,0.335993,0.789288,-0.146271,-0.110158,0.554049,0.712445,-0.146368,0.059334,0.558470,-0.073038,...,0.503232,-0.209546,-0.089230,0.222335,-0.027083,0.609238,Normal,P8,FP-E,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_AN_RF_AN_fold4_3.csv') #เปลี่ยนชื่อไฟล์